# DEIMOS SlitMask Slurping

In [175]:
# imports
from importlib import reload
import os
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table

from pypeit.spectrographs import util as spec_util
from pypeit import specobjs
from pypeit import slittrace

In [138]:
# Use a science image for rotation on sky!
maskdef_file = os.path.join(os.getenv('PYPEIT_DEV'), 'RAW_DATA', 'keck_deimos',
                            '830G_M_8500', 'DE.20100913.22358.fits.gz')

In [139]:
hdul = fits.open(maskdef_file)

# Spectrograph

In [2]:
spectro = spec_util.load_spectrograph('keck_deimos')

In [39]:
det = 3

In [46]:
det_par = spectro.get_detector_par(hdul, det=3)

# Load a SlitMask

In [4]:
slitmask = spectro.get_slitmask(maskdef_file)

> /home/xavier/Projects/PypeIt/pypeit/spectrographs/slitmask.py(195)__init__()
-> try:
(Pdb) c


## Fuss with objects

In [6]:
slitmask.objects

array([[ 6.54042000e+05,  6.92066000e+05,  3.52335663e+02,
        -2.90820278e+00],
       [ 6.54042000e+05,  6.92067000e+05,  3.52202883e+02,
        -3.10648333e+00],
       [ 6.54042000e+05,  6.92068000e+05,  3.52368979e+02,
        -2.96066389e+00],
       [ 6.54042000e+05,  6.92069000e+05,  3.52350433e+02,
        -3.12619444e+00],
       [ 6.54042000e+05,  6.92070000e+05,  3.52244483e+02,
        -3.20321667e+00],
       [ 6.54042000e+05,  6.92071000e+05,  3.52517675e+02,
        -2.94849444e+00],
       [ 6.54043000e+05,  6.92072000e+05,  3.52381317e+02,
        -3.18473333e+00],
       [ 6.54042000e+05,  6.92073000e+05,  3.52562283e+02,
        -3.05473611e+00],
       [ 6.54044000e+05,  6.92074000e+05,  3.52284483e+02,
        -3.03282222e+00],
       [ 6.54045000e+05,  6.92075000e+05,  3.52372329e+02,
        -3.16441944e+00],
       [ 6.54046000e+05,  6.92076000e+05,  3.52225958e+02,
        -2.98875556e+00],
       [ 6.54047000e+05,  6.92077000e+05,  3.52335279e+02,
      

In [13]:
obj_maskdef_id = slitmask.objects[:,0].astype(int)
obj_maskdef_id

array([654042, 654042, 654042, 654042, 654042, 654042, 654043, 654042,
       654044, 654045, 654046, 654047, 654048, 654049, 654050, 654051,
       654052, 654053, 654054, 654055, 654056, 654057, 654058, 654059,
       654060, 654061, 654062, 654063, 654064, 654065, 654066, 654067,
       654068, 654069, 654070, 654071, 654072, 654073, 654074, 654075,
       654076, 654077, 654078, 654079, 654080, 654081, 654082, 654083,
       654084, 654085, 654086, 654087, 654088, 654089, 654090, 654091,
       654092, 654093, 654094, 654095, 654096, 654097, 654098, 654099,
       654100, 654101, 654102, 654103, 654104, 654105, 654106, 654107,
       654108, 654109, 654110, 654111, 654112, 654113, 654114, 654115,
       654116, 654117, 654118, 654119, 654120, 654121, 654122, 654123,
       654124, 654125, 654126, 654127, 654128, 654129, 654130, 654131,
       654132, 654133, 654134, 654135, 654136, 654137, 654138, 654139,
       654140, 654141, 654142, 654143, 654144, 654145, 654146, 654147])

In [16]:
obj_coords = SkyCoord(ra=slitmask.objects[:,2], dec=slitmask.objects[:,3], unit='deg')

In [18]:
obj_coords[0:5]

<SkyCoord (ICRS): (ra, dec) in deg
    [(352.3356625 , -2.90820278), (352.20288333, -3.10648333),
     (352.36897917, -2.96066389), (352.35043333, -3.12619444),
     (352.24448333, -3.20321667)]>

In [88]:
obj_tbl = Table(hdul['ObjectCat'].data)
obj_tbl

ObjectId,OBJECT,RA_OBJ,DEC_OBJ,RADECSYS,EQUINOX,MJD_OBS,mag,pBand,RadVel,MajAxis,ObjClass
int32,str68,float64,float64,str8,float64,float64,float32,str6,float64,float32,str20
692066,LBG0,352.3356625,-2.90820278,FK5,2000.0,0.0,24.625,z,0.0,0.0,Program_Target
692067,LBG1,352.20288333,-3.10648333,FK5,2000.0,0.0,24.88,z,0.0,0.0,Program_Target
692068,LBG2,352.36897917,-2.96066389,FK5,2000.0,0.0,24.691,z,0.0,0.0,Program_Target
692069,LBG3,352.35043333,-3.12619444,FK5,2000.0,0.0,24.64,z,0.0,0.0,Program_Target
692070,LBG4,352.24448333,-3.20321667,FK5,2000.0,0.0,24.983,z,0.0,0.0,Program_Target
692071,LBG5,352.517675,-2.94849444,FK5,2000.0,0.0,25.003,z,0.0,0.0,Program_Target
692072,LBG6,352.38131667,-3.18473333,FK5,2000.0,0.0,25.109,z,0.0,0.0,Program_Target
692073,qso0,352.56228333,-3.05473611,FK5,2000.0,0.0,19.449,z,0.0,0.0,Program_Target
692074,qso1,352.28448333,-3.03282222,FK5,2000.0,0.0,21.759,z,0.0,0.0,Program_Target


## Parse slits a bit

In [21]:
slitmask.onsky[0:5]

array([[352.35030246,  -3.12600665,   7.67500019,   1.        ,
        -35.        ],
       [352.38105223,  -3.18435512,  21.5359993 ,   1.        ,
        -35.        ],
       [352.28461572,  -3.03300991,   8.77099991,   1.        ,
        -35.        ],
       [352.3723985 ,  -3.16451715,   8.30200005,   1.        ,
        -35.        ],
       [352.22562194,  -2.98827452,  10.39299965,   1.        ,
        -35.        ]])

In [20]:
slit_coords = SkyCoord(ra=slitmask.onsky[:,0], dec=slitmask.onsky[:,1], unit='deg')
slit_coords[0:5]

<SkyCoord (ICRS): (ra, dec) in deg
    [(352.35030246, -3.12600665), (352.38105223, -3.18435512),
     (352.28461572, -3.03300991), (352.3723985 , -3.16451715),
     (352.22562194, -2.98827452)]>

In [22]:
slit_pa = slitmask.onsky[:,-1]
slit_pa

array([-35., -35., -35., -35., -35., -35., -35., -35., -35., -35., -35.,
       -35., -35., -35., -35., -35., -35., -35., -35., -35., -35., -35.,
       -35., -35., -35., -35., -35., -35., -35., -35., -35., -35., -35.,
       -35., -35., -35., -35., -35., -35., -35., -35., -35., -35., -35.,
       -35., -35., -35., -35., -35., -35., -35., -35., -35., -35., -35.,
       -35., -35., -35., -35., -35., -35., -35., -35., -35., -35., -35.,
       -35., -35., -35., -35., -35., -35., -35., -35., -35., -35., -35.,
       -35., -35., -35., -35., -35., -35., -35., -35., -35., -35., -35.,
       -35., -35., -35., -35., -35., -35., -35., -35., -35., -35., -35.,
       -35., -30., -30., -30., -30., -35., -35.])

In [30]:
slitmask.slitid

array([654042, 654043, 654044, 654045, 654046, 654047, 654048, 654049,
       654050, 654051, 654052, 654053, 654054, 654055, 654056, 654057,
       654058, 654059, 654060, 654061, 654062, 654063, 654064, 654065,
       654066, 654067, 654068, 654069, 654070, 654071, 654072, 654073,
       654074, 654075, 654076, 654077, 654078, 654079, 654080, 654081,
       654082, 654083, 654084, 654085, 654086, 654087, 654088, 654089,
       654090, 654091, 654092, 654093, 654094, 654095, 654096, 654097,
       654098, 654099, 654100, 654101, 654102, 654103, 654104, 654105,
       654106, 654107, 654108, 654109, 654110, 654111, 654112, 654113,
       654114, 654115, 654116, 654117, 654118, 654119, 654120, 654121,
       654122, 654123, 654124, 654125, 654126, 654127, 654128, 654129,
       654130, 654131, 654132, 654133, 654134, 654135, 654136, 654137,
       654138, 654139, 654140, 654141, 654142, 654143, 654144, 654145,
       654146, 654147], dtype=int32)

## Slit Object coords

In [103]:
obj_slit_coords = []
for obj_mid in obj_maskdef_id:
    # Get the slit index
    idx = np.where(slitmask.slitid == obj_mid)[0]
    if len(idx) != 1:
        import pdb; pdb.set_trace()
    idx = idx[0]
    # Fill
    obj_slit_coords.append(slit_coords[idx])
#
obj_slit_coords = SkyCoord(obj_slit_coords)

In [105]:
# Object space
slit_to_obj_sep = obj_slit_coords.separation(obj_coords)
slit_to_obj_sep.to('arcsec')

<Angle [7.85858316e+02, 5.34564674e+02, 5.99008676e+02, 8.23614439e-01,
        4.71104368e+02, 8.77731998e+02, 1.66051543e+00, 8.04056366e+02,
        8.26477069e-01, 4.31087722e-01, 2.11222199e+00, 1.23708389e+00,
        5.56701619e-01, 4.80963812e-01, 3.22332485e-01, 2.11817154e-01,
        1.43745139e+00, 1.44854137e-01, 8.83452156e-02, 1.33773013e-01,
        3.68709518e-01, 2.58159667e+00, 6.70968586e-01, 1.77475488e-01,
        4.80600106e-01, 7.48046912e-02, 1.47046554e+00, 2.79408645e-01,
        1.65384850e+00, 3.33441385e-01, 6.21834715e-01, 4.82633302e-01,
        6.13482134e-01, 1.26619834e-01, 3.34610186e-01, 4.86171419e-01,
        1.48470911e-01, 1.82517263e+00, 4.23485843e-01, 4.72342332e-01,
        2.69065906e+00, 2.80997251e+00, 8.76911628e-01, 1.27693418e+00,
        1.81475081e-01, 2.53825577e-01, 1.35838544e-02, 1.00377334e-01,
        2.49126764e+00, 6.59451504e-01, 2.16429422e+00, 4.27628664e-01,
        1.45006088e-01, 1.25996010e+00, 2.48005914e-01, 2.388417

In [38]:
slit_to_obj_pa = obj_slit_coords.position_angle(obj_coords)
slit_to_obj_pa.to('deg').value

array([356.15950338, 277.55104694,   6.43615762, 145.16755108,
       233.83940582,  43.28024658, 145.08078339,  71.39726393,
       324.84000232, 324.68369521, 145.07162803, 145.11096187,
       324.75968616, 324.71718506, 145.43269819, 324.36437905,
       145.10248244, 145.95368152, 146.56842229, 323.96748336,
       145.37532157, 324.94955598, 324.79487809, 324.23094617,
       145.28775014, 146.81413651, 145.09475798, 145.49566375,
       145.08263585, 324.60078731, 145.22405196, 145.29205912,
       324.78188818, 146.0902517 , 145.41272465, 324.7239133 ,
       324.08182589, 324.93033257, 324.67782808, 324.71336495,
       145.05503694, 324.95427162, 145.16255158, 324.89973429,
       145.75368001, 324.45674172, 314.82587242, 323.62588845,
       145.05799362, 145.21156225, 145.06513897, 324.67871222,
       145.94104768, 145.10742661, 324.45011543, 324.946431  ,
       324.88986757, 145.08134768, 324.69453109, 324.68775799,
       145.27810876, 145.08337324, 145.15866609, 324.79

# Load PypeIt Slits

In [9]:
slits_file = os.path.join(os.getenv('PYPEIT_DEV'), 'REDUX_OUT', 'keck_deimos',
                            '830G_M_8500', 'Masters', 'MasterSlits_A_1_03.fits.gz')

In [11]:
slits = slittrace.SlitTraceSet.from_file(slits_file)

[INFO]    :: datamodel.py 1397 from_file() - Loading SlitTraceSet from /home/xavier/Projects/PypeIt-development-suite/REDUX_OUT/keck_deimos/830G_M_8500/Masters/MasterSlits_A_1_03.fits.gz


In [12]:
slits.maskdef_id

array([654130, 654053, 654103, 654083, 654102, 654136, 654076, 654108,
       654140, 654093, 654092, 654061, 654129, 654078, 654141, 654069,
       654055, 654065, 654051, 654060, 654082, 654098, 654044, 654090,
       654097], dtype=int32)

In [14]:
for maskdef in slits.maskdef_id:
    assert maskdef in obj_maskdef_id

# Load a SpecObjs

In [23]:
specobjs_file = os.path.join(os.getenv('PYPEIT_DEV'), 'REDUX_OUT', 'keck_deimos',
                            '830G_M_8500', 'Science', 'spec1d_DE.20100913.22358-CFHQS1_DEIMOS_2010Sep13T061231.334.fits')

In [24]:
sobjs = specobjs.SpecObjs.from_fitsfile(specobjs_file)

## Fuss with one -- 496 is bright and well offset from slit center

In [121]:
sobjs.NAME

array(['SPAT0476-SLIT0496-DET03', 'SPAT0660-SLIT0622-DET03',
       'SPAT0858-SLIT0863-DET03', 'SPAT1119-SLIT1124-DET03',
       'SPAT1246-SLIT1229-DET03', 'SPAT1636-SLIT1632-DET03'], dtype='<U23')

### Grab MASKDEF_ID

In [122]:
sobj = sobjs[0]
sobj.NAME

'SPAT0476-SLIT0496-DET03'

In [123]:
sobj.SLITID

496

In [124]:
sobj.MASKDEF_ID = slits.maskdef_id[slits.spat_id == sobj.SLITID][0]
sobj.MASKDEF_ID

654076

In [125]:
objids = slitmask.objects[:,1].astype(int)
ObjectID = objids[obj_maskdef_id == sobj.MASKDEF_ID][0]  # Allow for more than 1!!
ObjectID

692106

In [126]:
obj_tbl[obj_tbl['ObjectId'] == ObjectID]

ObjectId,OBJECT,RA_OBJ,DEC_OBJ,RADECSYS,EQUINOX,MJD_OBS,mag,pBand,RadVel,MajAxis,ObjClass
int32,str68,float64,float64,str8,float64,float64,float32,str6,float64,float32,str20
692106,ero89,352.27471667,-3.09223056,FK5,2000.0,0.0,16.135,z,0.0,0.0,Program_Target


### Expected offset

In [127]:
oidx = np.where(slitmask.objects[:,1].astype(int) == ObjectID)[0][0]
oidx

40

In [135]:
sidx = np.where(slitmask.slitid == sobj.MASKDEF_ID)[0][0]
sidx

34

In [128]:
slit_to_obj_sep[oidx].to('arcsec')

<Angle 2.69065906 arcsec>

### Actual

In [156]:
dpix = sobj.SPAT_PIXPOS - sobj.SLITID
dpix

-19.7769676707685

In [157]:
darcsec = dpix * det_par['platescale']
darcsec

-2.343570668986067

### PA!

In [158]:
slit_to_obj_pa[oidx].to('deg').value

145.05503694069927

In [159]:
slit_pa[sidx]

-35.0

In [160]:
hdul[0].header['ROTPOSN'] + 90    # SKY PA 

-29.981404409999996

In [161]:
posx = hdul[0].header['ROTPOSN'] + 90 
if posx < 0.:
    posx += 360.
posx

330.01859559

### Are we closer to posx or its complement?

In [162]:
if posx > 180.:
    comp_posx = posx - 180.
else:
    comp_posx = posx + 180.

In [163]:
imin = np.argmin(np.abs(slit_to_obj_pa[oidx].to('deg').value - np.array([posx, comp_posx])))
imin

1

In [176]:
sign = -1 if imin == 1 else 1
sign

-1

In [165]:
assert np.sign(dpix) == sign

## Need to worry about telescope offsets..

----

In [169]:
maskblu = Table(hdul['MaskBlu'].data)

In [170]:
maskblu

BluId,DesId,BluName,BluPId,BluCreat,BluDate,LST_Use,Date_Use,TeleId,RefrId,AtmTempC,AtmPres,AtmHumid,AtmTTLap,RefWave,DistId,guiname,millseq,status,loc,stamp
int32,int32,str68,int32,str68,str19,float32,str19,int32,int32,float32,float32,float32,float32,float32,int32,str8,str2,int16,str1,str19
8044,8073,CFHQS1,260,DSIMULATOR -- 02Jul25,2010-08-25T23:29:00,21.987,2010-09-12T00:00:00,1,0,0.0,615.0,0.4,0.0065,950.0,0,CFHQS1B,,2,,2010-08-25T14:30:00


In [171]:
hdul.info()

Filename: /home/xavier/Projects/PypeIt-development-suite/RAW_DATA/keck_deimos/830G_M_8500/DE.20100913.22358.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     720   ()      
  1  Video_Input_2    1 ImageHDU       360   (2140, 4096)   int16 (rescales to uint16)   
  2  Video_Input_4    1 ImageHDU       360   (2140, 4096)   int16 (rescales to uint16)   
  3  Video_Input_6    1 ImageHDU       360   (2140, 4096)   int16 (rescales to uint16)   
  4  Video_Input_8    1 ImageHDU       360   (2140, 4096)   int16 (rescales to uint16)   
  5  Video_Input_10    1 ImageHDU       360   (2140, 4096)   int16 (rescales to uint16)   
  6  Video_Input_12    1 ImageHDU       360   (2140, 4096)   int16 (rescales to uint16)   
  7  Video_Input_14    1 ImageHDU       360   (2140, 4096)   int16 (rescales to uint16)   
  8  Video_Input_16    1 ImageHDU       360   (2140, 4096)   int16 (rescales to uint16)   
  9  ObjectCat     1 BinTableHDU     55   112R 

In [172]:
maskdes = Table(hdul['MaskDesign'].data)

In [173]:
maskdes

DesId,DesName,DesPId,DesCreat,DesDate,DesNslit,DesNobj,ProjName,INSTRUME,MaskType,RA_PNT,DEC_PNT,RADEPNT,EQUINPNT,PA_PNT,DATE_PNT,LST_PNT,uid,stamp
int32,str68,int32,str68,str19,int32,int32,str68,str68,str68,float64,float64,str8,float32,float64,str23,float32,int16,str19
8073,CFHQS1,260,Dsimulator: Ver 0.0b,2010-08-25T23:29:00,106,114,Overdensity around QSO,DEIMOS,???,352.37082786,-3.0512015,FK5,2000.0,-30.0,2010-09-12T00:00:00.000,-22.5,38,2010-08-25T14:30:00


In [178]:
sobjs.DET

array([3, 3, 3, 3, 3, 3])